<a href="https://colab.research.google.com/github/gtejaswi13/bayeslab_prompt_assignment/blob/main/prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 2.5 MB/s eta 0:00:00


In [8]:
import os
from google.colab import userdata
groq_api_key = userdata.get('groq_api')
os.environ["GROQ_API_KEY"] = groq_api_key

In [10]:
!pip install -q langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 6.9 MB/s eta 0:00:00


In [11]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [13]:
!pip install -q litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.5 MB/s eta 0:00:00


In [14]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [16]:
def process_review(review_text):
    prompt = prompt_template.format(review_text=review_text)  # Format the string here
    response = get_response(prompt)
    return response

In [19]:
import gradio as gr
from litellm import completion
import os

# Set up API key for Groq
os.environ["GROQ_API_KEY"] = "groq_api"

# Define the LLM settings
def get_response(prompt):
    model = "groq/llama3-70b-8192"
    messages = [{"role": "user", "content": prompt}]

    # Call the model to get responses
    response = completion(
        model=model,
        messages=messages
    )
    return response['choices'][0]['message']['content']


# Define a prompt template for extracting customer reviews
prompt_template = """
You are an advanced AI assistant specialized in extracting structured information from customer reviews. Your task is to analyze the provided customer review and extract the following details in a clear and structured manner:

1. **Customer Details**:
   - Name (if available).
   - Customer ID (if mentioned).

2. **Product Information**:
   - Name of the product.
   - Key features or description of the product mentioned in the review.

3. **Company Information**:
   - Name of the company associated with the product.

4. **Sentiment Analysis**:
   - Overall sentiment of the customer: Positive, Negative, or Neutral.

5. **Feedback Analysis**:
   - Provide a detailed description of the positive aspects of the product, emphasizing features, performance, and any unique characteristics mentioned.
   - Provide a detailed description of the negative aspects of the product, highlighting issues, shortcomings, or any dissatisfaction expressed by the customer.

Please ensure your response is thorough and precise, with detailed insights extracted directly from the review text.

Here is the customer review:

Review: {review_text}
"""

# Define the main processing function
def process_review(review_text):
    prompt = prompt_template.format(review_text=review_text)
    response = get_response(prompt)
    return response

# Batch processing for multiple reviews
def batch_process(files):
    results = []
    for file in files:
        with open(file.name, 'r') as f:
            text = f.read()
            result = process_review(text)
            results.append({"file": file.name, "analysis": result})
    return results

def display_results(results):
    return "\n\n".join([
        f"File: {result['file']}\nAnalysis: {result['analysis']}"
        for result in results
    ])

# Gradio demo
def demo_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## Customer Review Extraction and Analysis")

        with gr.Row():
            file_input = gr.File(label="Upload Review Files", file_types=[".txt"], file_count="multiple")
            submit_button = gr.Button("Analyze")

        output = gr.Textbox(label="Analysis Results", lines=20)

        def analyze(files):
            results = batch_process(files)
            return display_results(results)

        submit_button.click(
            fn=analyze,
            inputs=[file_input],
            outputs=[output]
        )

    return demo

# Run the demo
if __name__ == "__main__":
    demo = demo_interface()
    demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f179ff95b3bdeaa19.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
